In [1]:
import pandas as pd
import csv
from pandas.core.frame import DataFrame
import numpy as np
from collections import defaultdict

In [2]:
#EDITABLE - merge multiple instapage files into one
#note: edit "Folder_Path":this is the path where you put insta files in ; "SaveFile_Path"; "SaveFile_Name"
'''
import os
import codecs

Folder_Path = 'D:\Work\TappedIn\data source leaking check\Instapage'          
SaveFile_Path = 'D:\Work\TappedIn\data source leaking check\Instapage'       
SaveFile_Name = 'insta_all.csv'              
    
file_list =os.listdir(Folder_Path)
#Read the first csv file including header


df = pd.read_csv(Folder_Path +'/'+ file_list[0],error_bad_lines=False)   #UTF-8 by default
df.to_csv(SaveFile_Path+'/'+ SaveFile_Name,encoding="utf_8",index=False)

#iterate csv name in the list and append them
for i in range(1,len(file_list)):
    #df = pd.read_csv(Folder_Path + '/'+ file_list[i],error_bad_lines=False)
    df = pd.read_csv(Folder_Path + '/'+ file_list[i])
    #print(file_list[i])
    #print(df.shape)
    df.to_csv(SaveFile_Path+'/'+ SaveFile_Name,encoding="utf_8",index=False, header=False, mode='a+')
'''

'\nimport os\nimport codecs\n\nFolder_Path = \'D:\\Work\\TappedIn\\data source leaking check\\Instapage\'          \nSaveFile_Path = \'D:\\Work\\TappedIn\\data source leaking check\\Instapage\'       \nSaveFile_Name = \'insta_all.csv\'              \n    \nfile_list =os.listdir(Folder_Path)\n#Read the first csv file including header\n\n\ndf = pd.read_csv(Folder_Path +\'/\'+ file_list[0],error_bad_lines=False)   #UTF-8 by default\ndf.to_csv(SaveFile_Path+\'/\'+ SaveFile_Name,encoding="utf_8",index=False)\n\n#iterate csv name in the list and append them\nfor i in range(1,len(file_list)):\n    #df = pd.read_csv(Folder_Path + \'/\'+ file_list[i],error_bad_lines=False)\n    df = pd.read_csv(Folder_Path + \'/\'+ file_list[i])\n    #print(file_list[i])\n    #print(df.shape)\n    df.to_csv(SaveFile_Path+\'/\'+ SaveFile_Name,encoding="utf_8",index=False, header=False, mode=\'a+\')\n'

In [3]:
#EDITABLE - read data
#note: edit the content between "" as the files addresses
leads_data = []
with open(r"D:\Work\TappedIn\data source leaking check\CRM_-_Leads_-_New.csv") as csvfile: # put in leads file address
    csv_reader = csv.reader(csvfile)
    leads_header = next(csv_reader)  
    for row in csv_reader: 
        leads_data.append(row)
leads = pd.DataFrame(leads_data[0:], columns=leads_header) 

contacts_data = []
with open(r"D:\Work\TappedIn\data source leaking check\CRM - Contacts.csv") as csvfile: # put in leads file address
    csv_reader = csv.reader(csvfile)
    contacts_header = next(csv_reader)  
    for row in csv_reader: 
        contacts_data.append(row)
contacts = pd.DataFrame(contacts_data[0:], columns=contacts_header) 

script_log_data = []
with open(r"D:\Work\TappedIn\data source leaking check\script log.csv") as csvfile: # put in leads file address
    csv_reader = csv.reader(csvfile)
    script_log_header = next(csv_reader)  
    for row in csv_reader: 
        script_log_data.append(row)
script_log = pd.DataFrame(script_log_data[0:], columns=script_log_header) 

instapage_data = []
with open(r"D:\Work\TappedIn\data source leaking check\Instapage\insta_all.csv") as csvfile: # put in leads file address
    csv_reader = csv.reader(csvfile)
    instapage_header = next(csv_reader)  
    for row in csv_reader: 
        instapage_data.append(row)
instapage = pd.DataFrame(instapage_data[0:], columns=instapage_header) 

In [4]:
print(instapage.shape)
print(script_log.shape)
print(leads.shape)
print(contacts.shape)

(65017, 15)
(72272, 12)
(114455, 59)
(82539, 62)


# 1. pre-processing
## 1.1 pre-processing in general: 
- Script log: Extract 'Visit ID' and 'Wifi Login ID' from 'Query String'
- Fill in blanks with 'Unknown'
- Extract insta, script and unknown data in Leads Only -  For both Leads and Contacts, filtered out data sources that are not in the data flow(e.g jot form etc,.); 
- Create new identifier in insta,script, leads called ['Date_Email_location_frequency']
- Extract certain date range from insta, script, Leads(this script only for data from 4/15/2019)

In [5]:
#1. Script log: Extract 'Visit ID' and 'Wifi Login ID' from 'Query String'
q_str=script_log['Query String']
v_lst=[]
w_lst=[]

for i in q_str:
    if 'VisitId' in i:
        for j in i.split('&'):
            if 'VisitId' in j:
                v_lst.append(j.split('=')[-1])
            else:
                continue
    else:
        v_lst.append('Unknown')

for i in q_str:
    if 'WiFiLogInId' in i:
        for j in i.split('&'):
            if 'WiFiLogInId' in j:
                w_lst.append(j.split('=')[-1])
            else:
                continue
    else:
        w_lst.append('Unknown')

In [6]:
script_log['Visit ID'] = v_lst
script_log['Wifi Login ID'] = w_lst

In [7]:
#save the new script log file with seperated visit ID
script_log.to_csv('script_log_seperate_visitID.csv',index=False)

In [8]:
#fill in blank with "unknown"
leads.replace(to_replace='',value='Unknown',inplace=True)
contacts.replace(to_replace='',value='Unknown',inplace=True)
script_log.replace(to_replace='',value='Unknown',inplace=True)
instapage.replace(to_replace='',value='Unknown',inplace=True)

In [9]:
#convert date columns in to timestamp format
script_log['CST'] = pd.to_datetime(script_log['CST']).dt.date
leads['Created Time'] = pd.to_datetime(leads['Created Time']).dt.date
contacts['Created Time'] = pd.to_datetime(contacts['Created Time']).dt.date
instapage['Date'] = pd.to_datetime(instapage['Date']).dt.date

In [10]:
#sort by date
script_log.sort_values(by='CST',ascending=False, inplace=True)
leads.sort_values(by='Created Time',ascending=False, inplace=True)
contacts.sort_values(by='Created Time',ascending=False, inplace=True)
instapage.sort_values(by='Date',ascending=False, inplace=True)

In [11]:
#3.Extract insta, script and unknown data in Leads
leads = leads[['LEADID','Email','Created Time', 'Location', 'TappedIn Data Source','Visit Id','WiFi Log In Id','Location Guest','Flag']]
#leads_new['TappedIn Data Source'].unique()
leads_data_source_filter=['WiFi On Tap - Instapage','Instapage','WiFi On Tap - TappedIn Redirect - Presence Orb',
                       'WiFi On Tap - TappedIn Redirect - Precense Orb','Justin App','Tapped Innovations','Unknown']
leads=leads.loc[leads['TappedIn Data Source'].isin(leads_data_source_filter)]
leads.shape

(100222, 9)

In [12]:
#4. Create new identifier in insta,script and leads called ['Date_Email_location_frequency']
script_log['Date_Email_Location']=script_log['CST'].map(str)+'_'+script_log['Email']+'_'+script_log['Location Id']
instapage['Date_Email_Location']= instapage["Date"].map(str) +'_'+instapage["email"]+'_'+instapage['location']
leads['Date_Email_Location']=leads['Created Time'].map(str)+'_'+leads['Email']+'_'+leads['Location']

In [13]:
def freq(record):
    record_count[record]+=1
    return str(record_count[record])

def count_record_freq(data):
    data['frequency'] = data.apply(lambda x:freq(x['Date_Email_Location']), axis=1)

In [14]:
record_count = defaultdict(int) #int() generate 0
count_record_freq(script_log)
record_count = defaultdict(int) #int() generate 0
count_record_freq(instapage)
record_count = defaultdict(int) #int() generate 0
count_record_freq(leads)

In [15]:
script_log['Date_Email_Location_Freq']=script_log['Date_Email_Location']+'_'+script_log['frequency']
instapage['Date_Email_Location_Freq']= instapage['Date_Email_Location']+'_'+instapage['frequency']
leads['Date_Email_Location_Freq']=leads['Date_Email_Location']+'_'+leads['frequency']

In [16]:
#5. Extract certain date range from insta, script, Leads and Contacts (this script only for data from 4/15/2019)
instapage_new=instapage.loc[instapage['Date']>pd.to_datetime('2019-04-14')].reset_index(drop=True)
script_log_new=script_log.loc[script_log['CST']>pd.to_datetime('2019-04-14')].reset_index(drop=True)
leads_new=leads.loc[leads['Created Time']>pd.to_datetime('2019-04-14')].reset_index(drop=True)
contacts_new=contacts.loc[contacts['Created Time']>pd.to_datetime('2019-04-14')].reset_index(drop=True)

In [17]:
print(instapage_new.shape)
print(script_log_new.shape)
print(leads_new.shape)
print(contacts_new.shape)

(38311, 18)
(38301, 17)
(74498, 12)
(41592, 62)


In [18]:
#save files
instapage_new.to_csv('instapage_new.csv',index=False)
script_log_new.to_csv('script_log_new.csv', index=False)
leads_new.to_csv('leads_new.csv', index=False)
#contacts_new.to_csv('contacts_new.csv', index=False)

## 1.2 Preprocessing for Leads
Figure out and filling out unknown sources in Leads(prepare for the data check in the next step)

#### Senerio 1: For leads records with unknown ['TappedIn Data Source'] & known ['Visit Id']

- Correct spelling: change "WiFi On Tap - TappedIn Redirect - Precense Orb" to "WiFi On Tap - TappedIn Redirect - Presence Orb"
- Records with ['Visit Id''] matched with Redirect Script['Visit id'] but not with Instapage['Visit id'] ---- filling out ['Tappedin Data Source'] with 'Filled Redirect Script'
- Records with ['Visit Id''] matched with both Redirect Script['Visit id'] and Instapage['Visit id'] ---- filling out ['Tappedin Data Source'] with 'Filled instapage'
- Records  with ['Visit Id'] matched with Instapage['Visit id'] but not with Redirect Script['Visit id'] ----- filling out ['Tappedin Data Source'] with 'Filled Instapage only'
- Rest of records would have ['Tappedin Data Source'] keep as 'Unknown'

In [19]:
leads_new.loc[leads_new['TappedIn Data Source']=='WiFi On Tap - TappedIn Redirect - Precense Orb', 
              ['TappedIn Data Source']] = 'WiFi On Tap - TappedIn Redirect - Presence Orb'

In [20]:
leads_unknown_source=leads_new.loc[leads_new['TappedIn Data Source']=='Unknown']
leads_2=leads_unknown_source.loc[(leads_unknown_source['Visit Id']!='Unknown') 
                                 & (leads_unknown_source['Visit Id']!='[VisitId]')]

In [21]:
script_visit_id=script_log_new['Visit ID'].tolist()
instapage_visit_id=instapage_new['Visit ID'].tolist()
leads_2_visit_id=leads_2['Visit Id'].tolist()

In [22]:
for i in leads_2.index:
    if ((leads_2['Visit Id'][i] in script_visit_id) and (leads_2['Visit Id'][i] not in instapage_visit_id)):
        leads_2['TappedIn Data Source'][i] = 'Filled Redirect Script'

C:\Users\Martin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
for i in leads_2.index:
    if ((leads_2['Visit Id'][i] in script_visit_id) and (leads_2['Visit Id'][i] in instapage_visit_id)):
        leads_2['TappedIn Data Source'][i] = 'Filled Instapage'

In [24]:
for i in leads_2.index:
    if ((leads_2['Visit Id'][i] not in script_visit_id) and (leads_2['Visit Id'][i] in instapage_visit_id)):
        leads_2['TappedIn Data Source'][i] = 'Filled Instapage(only)'

In [25]:
leads_2_index=leads_2.index.tolist()

In [26]:
for i in leads_2_index:
    leads_new['TappedIn Data Source'][i]=leads_2['TappedIn Data Source'][i]

#### Scenario 2: For leads records with unknown ['TappedIn Data Source'] & unknown ['Visit Id']
- Repeat the solution in Scenario 1, just change ['Visit ID'] with ['Date_Email_location_frequency']

In [27]:
leads_3=leads_unknown_source.loc[(leads_unknown_source['Visit Id']=='Unknown') 
                                 | (leads_unknown_source['Visit Id']=='[VisitId]')]

In [28]:
script_DELF=script_log_new['Date_Email_Location_Freq'].tolist()
instapage_DELF=instapage_new['Date_Email_Location_Freq'].tolist()
leads_3_DELF=leads_3['Date_Email_Location_Freq'].tolist()

In [29]:
for i in leads_3.index:
    if ((leads_3['Date_Email_Location_Freq'][i] in script_DELF) and (leads_3['Date_Email_Location_Freq'][i] not in instapage_DELF)):
        leads_3['TappedIn Data Source'][i] = 'Filled Redirect Script'

In [30]:
for i in leads_3.index:
    if ((leads_3['Date_Email_Location_Freq'][i] in script_DELF) and (leads_3['Date_Email_Location_Freq'][i] in instapage_DELF)):
        leads_3['TappedIn Data Source'][i] = 'Filled Instapage'

In [31]:
for i in leads_3.index:
    if ((leads_3['Date_Email_Location_Freq'][i] not in script_DELF) and (leads_3['Date_Email_Location_Freq'][i] in instapage_DELF)):
        leads_3['TappedIn Data Source'][i] = 'Filled Instapage (only)'

In [32]:
leads_3_index=leads_3.index.tolist()
for i in leads_3_index:
    leads_new['TappedIn Data Source'][i]=leads_3['TappedIn Data Source'][i]

In [33]:
#save file
leads_new.to_csv('leads_new(preprocessed).csv',index=False)

## 1.3 Preprocessing for CRM Contacts
Extract useful info in CRM Contacts dataset

In [34]:
#Extract useful info from Contacts
contacts_new = contacts_new[['CONTACTID','Email','Created Time', 'Location', 'Email Opt Out','User Type','User ID','Data Source']]

In [35]:
contacts_new['Data Source']='CRM Contacts'

In [36]:
contacts_new.shape

(41592, 8)

In [37]:
contacts_new.to_csv('contacts_new.csv', index=False)

# 2. Data check I (for redirect script, instapage to Leads)
## 2.1: script ->instapage

#### 2.1.1 Records that are transferred = part 1 + part 2 
- part 1: (for insta records with  ['Visit ID'] in insta != 'Unknown' and != '[Visit ID]', use ['VisitID'] as identifier): 
    - Transferred records are with common ['VisitID'] in both script and instapage
- part 2: (for insta records with  ['Visit ID'] in insta == 'Unknown' or == '[Visit ID]', use ['Date_Email_location_frequency'] as identifier]): 
    - Transferred records are with common ['Date_Email_location_frequency'] in both script and instapage
 
#### 2.1.2 Creat new columns (in each of the data source files)
During 2.1.1, to help with visualization later on, we also create some new columns in each of the data source files, which including:
- Script To Insta Based On Visit ID
- Script To Insta Based On Date_Email_Location_Freq ID
- Script TO Insta

In [38]:
#read data (After first time of running this script, you can start from here directly)
#HOW TO USE: 1. delete "#" before each line to make them excutable
#leads_new_test= pd.read_csv("leads_new(preprocessed).csv")
#script_log_new= pd.read_csv("script_log_new.csv")
#instapage_new= pd.read_csv("instapage_new.csv")

In [39]:
#2.1.1 The number of records that are transferred = part 1 + part 2
#part 1: if 'VisitID' != unknown and 'VisitID' != [visitID]
insta_pt1=instapage_new.loc[(instapage_new['Visit ID']!='Unknown') & (instapage_new['Visit ID']!='[VisitId]')]

In [40]:
insta_pt1_lst=insta_pt1['Visit ID'].tolist()
script_vid_lst=script_log_new['Visit ID'].tolist()

In [41]:
list3 = [] #common factor - target list
list4 = [] #in script but not in insta
list5 = [] #in insta but not in script

#common factor in both lists
list3 = [x for x in insta_pt1_lst if x in script_vid_lst]

#factors in script but not in insta(possible reason: 1.insta visit id missed 2. didn't transfer from script to insta)
list4 = [x for x in script_vid_lst if x not in insta_pt1_lst] 

#factors in insta but not in script 
list5 = [x for x in insta_pt1_lst if x not in script_vid_lst] 
        
print(len(list3))
print(len(list4))
print(len(list5))

20091
18567
295


In [42]:
#craete 'Script To Insta Based On Visit ID' column in script log dataset 
script_log_index_lst=[0] * script_log_new.shape[0]
for i in script_log_new.index:
    if script_log_new['Visit ID'][i] in list3:
        script_log_index_lst[i]=1
script_log_new['Script To Insta Based On Visit ID']=script_log_index_lst

In [43]:
#craete 'Script To Insta Based On Visit ID' column in instapage dataset
instapage_index_lst=[0] * instapage_new.shape[0]
for i in instapage_new.index:
    if instapage_new['Visit ID'][i] in list3:
        instapage_index_lst[i]=1
instapage_new['Script To Insta Based On Visit ID']=instapage_index_lst

In [44]:
#filled 'NA' in Leads['Script To Insta Based On Visit ID']
leads_new['Script To Insta Based On Visit ID']='NA'

In [45]:
#part 2: if ['Visit ID'] in insta == 'Unknown' or == '[Visit ID]'
insta_pt2=instapage_new.loc[(instapage_new['Visit ID']=='Unknown') | (instapage_new['Visit ID']=='[VisitId]')]

In [46]:
insta_pt2_lst=insta_pt2['Date_Email_Location_Freq'].tolist()
script_DELF_lst=script_log_new['Date_Email_Location_Freq'].tolist()

In [47]:
list0 = [] #common factor -target list
list1 = [] #in script but not in insta
list2 = [] #in insta but not in script

#common factor in both lists
list0 = [x for x in insta_pt2_lst if x in script_DELF_lst]

#factors in script but not in insta(possible reason: 1.insta visit id missed 2. didn't transfer from script to insta)
list1 = [x for x in script_DELF_lst if x not in insta_pt2_lst] 

#factors in insta but not in script 
list2 = [x for x in insta_pt2_lst if x not in script_DELF_lst] 
        
print(len(list0))
print(len(list1))
print(len(list2))

7278
31023
10647


In [48]:
#craete 'Script To Insta Based On Date_Email_Location_Freq ID' column in script log dataset 
script_log_index_lst=[0] * script_log_new.shape[0]
for i in script_log_new.index:
    if script_log_new['Date_Email_Location_Freq'][i] in list0:
        script_log_index_lst[i]=1
script_log_new['Script To Insta Based On Date_Email_Location_Freq ID']=script_log_index_lst

In [49]:
#craete 'Script To Insta Based On Date_Email_Location_Freq ID' column in instapage dataset 
instapage_index_lst=[0] * instapage_new.shape[0]
for i in instapage_new.index:
    if instapage_new['Date_Email_Location_Freq'][i] in list0:
        instapage_index_lst[i]=1
instapage_new['Script To Insta Based On Date_Email_Location_Freq ID']=instapage_index_lst

In [50]:
#filled 'NA' in Leads['Script To Insta Based On Date_Email_Location_Freq ID']
leads_new['Script To Insta Based On Date_Email_Location_Freq ID']='NA'

In [51]:
#create ['Script To Insta'] in script log dataset
script_log_index_lst=[0]*script_log_new.shape[0]
for i in script_log_new.index:
    if (script_log_new['Script To Insta Based On Visit ID'][i]==1) or (script_log_new['Script To Insta Based On Date_Email_Location_Freq ID'][i]==1):
        script_log_index_lst[i]=1
script_log_new['Script To Insta']=script_log_index_lst

In [52]:
#create ['Script To Insta'] in instapage dataset
instapage_index_lst=[0]*instapage_new.shape[0]
for i in instapage_new.index:
    if (instapage_new['Script To Insta Based On Visit ID'][i]==1) or (instapage_new['Script To Insta Based On Date_Email_Location_Freq ID'][i]==1):
        instapage_index_lst[i]=1
instapage_new['Script To Insta']=instapage_index_lst

In [53]:
#fill in 'NA' to ['Script TO Insta'] in Leads dataset
leads_new['Script To Insta']='NA'

## 2.2 Instapage->Leads

#### 2.2.1 Records that are transferred
The leads records that are transferred from Instapage to Leads are with ['Tappedin Data Source']== 'Filled Instapage' or == 'WiFi On Tap - Instapage' or == 'Filled instapage (only)' or == 'instapage')

#### 2.2.2 Records that are not transferred
- S1 = insta records with known ['Visit ID'] that are not in leads(insta related) or insta records with unknown ['Visit ID']
- Considering leads records with ['Visit ID']='Unknown', If  S2= records in S1 with ['Date_Email_location_frequency'] can't be found in leads(insta related)['Date_Email_location_frequency'], records that are not transferred = S2

#### 2.2.3 Creat new columns (in each of the data source files)
During above process, to help with visualization later on, we also create some new columns in each of the data source files, which including:
- Insta To Leads Based On Visit ID
- Insta To Leads Based On Date_Email_Location_Freq ID
- Insta To Leads

In [54]:
leads_new['TappedIn Data Source'].unique()

array(['Filled Redirect Script', 'Unknown', 'Filled Instapage',
       'Filled Instapage(only)', 'WiFi On Tap - Instapage', 'Instapage',
       'WiFi On Tap - TappedIn Redirect - Presence Orb',
       'Tapped Innovations'], dtype=object)

In [55]:
#2.2.1 Records that are transferred
insta_2_leads_records=leads_new.loc[(leads_new['TappedIn Data Source']=='Filled Instapage') |
                                    (leads_new['TappedIn Data Source']=='WiFi On Tap - Instapage') |
                                    (leads_new['TappedIn Data Source']=='Filled Instapage(only)') |
                                    (leads_new['TappedIn Data Source']=='Instapage')]

In [56]:
#create ['Insta to Leads'] in leads dataset
leads_index_lst=[0]*leads_new.shape[0]
for i in leads_new.index:
    if (leads_new['TappedIn Data Source'][i]=='Filled Instapage') or (leads_new['TappedIn Data Source'][i]=='WiFi On Tap - Instapage') or (leads_new['TappedIn Data Source'][i]=='Filled Instapage(only)') or (leads_new['TappedIn Data Source'][i]=='Instapage'):
        leads_index_lst[i]=1
leads_new['Insta To Leads']=leads_index_lst

In [57]:
#Create ['Insta To Leads'] and filled 'NA' in script log dataset
script_log_new['Insta To Leads']='NA'

In [58]:
#2.2.2 Records that are not transferred
insta_2_leads_records_vid_lst= insta_2_leads_records['Visit Id'].tolist() #leads records (insta related) 
                                                                          #and have unknown and unknown id
instapage_new_known_id_lst=instapage_new.loc[(instapage_new['Visit ID']!='Unknown')&(instapage_new['Visit ID']!='[VisitId]')]['Visit ID'].tolist()

In [59]:
#factors in insta(with known id) but not in leads(insta related)
list7 = [x for x in instapage_new_known_id_lst if x not in insta_2_leads_records_vid_lst] 

In [60]:
lst=[]
for i in instapage_new.index:
    if (instapage_new['Visit ID'][i] in list7) or (instapage_new['Visit ID'][i]=='Unknown') or (instapage_new['Visit ID'][i]=='[VisitId]'):
        lst.append(i)
s1_insta_2_leads=instapage_new.iloc[lst]

In [61]:
s1_insta_2_leads_DELF=s1_insta_2_leads['Date_Email_Location_Freq'].tolist()
insta_2_leads_records_DELF=insta_2_leads_records['Date_Email_Location_Freq'].tolist()

In [62]:
#factors in S1 DELF but not in leads(insta related) DELF
list8 = [x for x in s1_insta_2_leads_DELF if x not in insta_2_leads_records_DELF] 

In [63]:
lst=[]
for i in instapage_new.index:
    if instapage_new['Date_Email_Location_Freq'][i] in list8:
        lst.append(i)
not_transferred_insta_2_leads_record=instapage_new.loc[lst]

In [64]:
#2.1.3 Creat new columns (in each of the data source files)
#create ['Insta To Leads'] in Instapage
instapage_index_lst=[1]*instapage_new.shape[0]
for i in instapage_new.index:
    if i in not_transferred_insta_2_leads_record.index.tolist():
        instapage_index_lst[i]=0
instapage_new['Insta To Leads']=instapage_index_lst

In [65]:
#create ['Insta To Leads Based On Visit ID'] in Instapage dataset
instapage_index_lst=[0]*instapage_new.shape[0]
for i in instapage_new.index:
    if (instapage_new['Insta To Leads'][i]==1) and (instapage_new['Visit ID'][i]!='Unknown') and (instapage_new['Visit ID'][i]!='[VisitId]'):
        instapage_index_lst[i]=1
instapage_new['Insta To Leads Based On Visit ID']=instapage_index_lst

In [66]:
#create ['Insta To Leads Based On Visit ID'] in Leads dataset 
insta_to_leads_based_on_vid_in_instaSet=instapage_new.loc[instapage_new['Insta To Leads Based On Visit ID']==1]['Visit ID'].tolist()
leads_index_lst=[0]*leads_new.shape[0]
for i in leads_new.index:
    if leads_new['Visit Id'][i] in insta_to_leads_based_on_vid_in_instaSet:
        leads_index_lst[i]=1
leads_new['Insta To Leads Based On Visit ID']=leads_index_lst

In [67]:
#create ['Insta To Leads Based On Visit ID'] in Script dataset and fill in with 'NA'
script_log_new['Insta To Leads Based On Visit ID']='NA'

In [68]:
#create['Insta To Leads Based On Date_Email_Location_Freq ID'] in instapage dataset
instapage_index_lst=[0]*instapage_new.shape[0]
for i in instapage_new.index:
    if (instapage_new['Insta To Leads'][i]==1) and ((instapage_new['Visit ID'][i]=='Unknown') or (instapage_new['Visit ID'][i]=='[VisitId]')):
        instapage_index_lst[i]=1
instapage_new['Insta To Leads Based On Date_Email_Location_Freq ID']=instapage_index_lst

In [69]:
#create ['Insta To Leads Based On Date_Email_Location_Freq ID'] in Leads dataset
insta_to_leads_based_on_wenyiID_in_instaSet=instapage_new.loc[instapage_new['Insta To Leads Based On Date_Email_Location_Freq ID']==1]['Visit ID'].tolist()
leads_index_lst=[0]*leads_new.shape[0]
for i in leads_new.index:
    if leads_new['Visit Id'][i] in insta_to_leads_based_on_wenyiID_in_instaSet:
        leads_index_lst[i]=1
leads_new['Insta To Leads Based On Date_Email_Location_Freq ID']=leads_index_lst

In [70]:
#create ['Insta To Leads Based On Date_Email_Location_Freq ID'] in Script dataset and fill in with 'NA'
script_log_new['Insta To Leads Based On Date_Email_Location_Freq ID']='NA'

## 2.3 Script->Leads

#### 2.3.1 Records that are transferred
The leads records that are transferred from Script to Leads are with ['Tappedin Data Source']=='WiFi On Tap - TappedIn Redirect - Presence Orb'' or == 'WiFi On Tap - TappedIn Redirect - Precense Orb' or == 'Redirect Script' or ==''Tapped Innovations')

#### 2.3.2 Records that are not transferred
- S1 = script records with known ['Visit ID'] that are not in leads(script related) or script records with unknown ['Visit ID']
- Considering leads records with ['Visit ID']='Unknown', If  S2= records in S1 with ['Date_Email_location_frequency'] can't be found in leads['Date_Email_location_frequency'], records that are not transferred = S2

#### 2.3.3 Creat new columns (in each of the data source files)
During above process, to help with visualization later on, we also create some new columns in each of the data source files, which including:
- Script To Leads Based On Visit ID
- Script To Leads Based On Date_Email_Location_Freq ID
- Script To Leads

In [71]:
leads_new['TappedIn Data Source'].unique()

array(['Filled Redirect Script', 'Unknown', 'Filled Instapage',
       'Filled Instapage(only)', 'WiFi On Tap - Instapage', 'Instapage',
       'WiFi On Tap - TappedIn Redirect - Presence Orb',
       'Tapped Innovations'], dtype=object)

In [72]:
#2.3.1 Records that are transferred
script_2_leads_records=leads_new.loc[(leads_new['TappedIn Data Source']=='Filled Redirect Script') |
                                    (leads_new['TappedIn Data Source']=='WiFi On Tap - TappedIn Redirect - Presence Orb') |
                                    (leads_new['TappedIn Data Source']=='Tapped Innovations')]

In [73]:
#create ['Script to Leads'] in leads dataset
leads_index_lst=[0]*leads_new.shape[0]
for i in leads_new.index:
    if (leads_new['TappedIn Data Source'][i]=='Filled Redirect Script') or (leads_new['TappedIn Data Source'][i]=='WiFi On Tap - TappedIn Redirect - Presence Orb') or (leads_new['TappedIn Data Source'][i]=='Tapped Innovations'):
        leads_index_lst[i]=1
leads_new['Script To Leads']=leads_index_lst

In [74]:
#create ['Script To Leads'] in instapage dataset
instapage_new['Script To Leads']='NA'

In [75]:
#2.3.2 Records that are not transferred
script_2_leads_records_vid_lst= script_2_leads_records['Visit Id'].tolist() #have unknown and unknown id
script_new_known_id_lst=script_log_new.loc[(script_log_new['Visit ID']!='Unknown')&(script_log_new['Visit ID']!='[VisitId]')]['Visit ID'].tolist()

In [76]:
#factors in script(with known id) but not in leads(script related)
list9 = [x for x in script_new_known_id_lst if x not in script_2_leads_records_vid_lst] 

In [77]:
lst=[]
for i in script_log_new.index:
    if (script_log_new['Visit ID'][i] in list9) or (script_log_new['Visit ID'][i]=='Unknown') or (script_log_new['Visit ID'][i]=='[VisitId]'):
        lst.append(i)
s1_script_2_leads=script_log_new.iloc[lst]

In [78]:
s1_script_2_leads_DELF=s1_script_2_leads['Date_Email_Location_Freq'].tolist()
script_2_leads_records_DELF=script_2_leads_records['Date_Email_Location_Freq'].tolist()

In [79]:
#factors in S1 DELF but not in leads(insta related) DELF
list10 = [x for x in s1_script_2_leads_DELF if x not in script_2_leads_records_DELF] 

In [80]:
lst=[]
for i in script_log_new.index:
    if script_log_new['Date_Email_Location_Freq'][i] in list10:
        lst.append(i)
not_transferred_script_2_leads_record=script_log_new.loc[lst]

In [81]:
#create ['Script To Leads'] in script dataset
script_index_lst=[1]*script_log_new.shape[0]
for i in script_log_new.index:
    if i in not_transferred_script_2_leads_record.index.tolist():
        script_index_lst[i]=0
script_log_new['Script To Leads']=script_index_lst

In [82]:
#create ['Script To Leads Based On Visit ID'] in script dataset
script_index_lst=[0]*script_log_new.shape[0]
for i in script_log_new.index:
    if (script_log_new['Script To Leads'][i]==1) and (script_log_new['Visit ID'][i]!='Unknown') and (script_log_new['Visit ID'][i]!='[VisitId]'):
        script_index_lst[i]=1
script_log_new['Script To Leads Based On Visit ID']=script_index_lst

In [83]:
#create ['Script To Leads Based On Visit ID'] in Leads dataset 
script_to_leads_based_on_vid_in_scriptSet=script_log_new.loc[script_log_new['Script To Leads Based On Visit ID']==1]['Visit ID'].tolist()
leads_index_lst=[0]*leads_new.shape[0]
for i in leads_new.index:
    if leads_new['Visit Id'][i] in script_to_leads_based_on_vid_in_scriptSet:
        leads_index_lst[i]=1
leads_new['Script To Leads Based On Visit ID']=leads_index_lst

In [84]:
#create ['Script To Leads Based On Visit ID'] in instapage, and fill in 'NA'
instapage_new['Script To Leads Based On Visit ID']='NA'

In [85]:
#create['Script To Leads Based On Date_Email_Location_Freq ID'] in Script dataset
script_index_lst=[0]*script_log_new.shape[0]
for i in script_log_new.index:
    if (script_log_new['Script To Leads'][i]==1) and ((script_log_new['Visit ID'][i]=='Unknown') or (script_log_new['Visit ID'][i]=='[VisitId]')):
        script_index_lst[i]=1
script_log_new['Script To Leads Based On Date_Email_Location_Freq ID']=script_index_lst

In [86]:
#create ['Script To Leads Based On Date_Email_Location_Freq ID'] in Leads dataset 
script_to_leads_based_on_wenyiID_in_scriptSet=script_log_new.loc[script_log_new['Script To Leads Based On Date_Email_Location_Freq ID']==1]['Visit ID'].tolist()
leads_index_lst=[0]*leads_new.shape[0]
for i in leads_new.index:
    if leads_new['Visit Id'][i] in script_to_leads_based_on_wenyiID_in_scriptSet:
        leads_index_lst[i]=1
leads_new['Script To Leads Based On Date_Email_Location_Freq ID']=leads_index_lst

In [87]:
#create ['Script To Leads Based On Date_Email_Location_Freq ID'] in Insta dataset and fill in with 'NA'
instapage_new['Script To Leads Based On Date_Email_Location_Freq ID']='NA'

## 2.4 Datasets Combination of Redirect script, Insta and Leads
Make 3 data source files into 1 so that we can make visuals in Power BI

In [88]:
#Add "Data Source" column in Script Log and Leads, unify text in Instapage's Data Source' column
script_log_new['Data Source']='Redirect Script'
leads_new['Data Source']='Leads'
instapage_new['Data Source']='Instapage'

In [89]:
#Add "Leads Data Source",'LEADID' "Location Guest "columns in Script Log and Instapage and fill with 'NA'
script_log_new['Leads Data Source']='NA'
instapage_new['Leads Data Source']='NA'
script_log_new['LEADID']='NA'
instapage_new['LEADID']='NA'
script_log_new['Location Guest']='NA'
instapage_new['Location Guest']='NA'

In [90]:
#Unify names 
script_log_new = script_log_new.rename({"CST":"Date","Location Id":"Location"}, axis='columns') 
instapage_new=instapage_new.rename({"location":"Location","email":"Email"},axis='columns')
leads_new=leads_new.rename({"Created Time":"Date","Visit Id":"Visit ID","TappedIn Data Source":"Leads Data Source"},axis='columns')

In [91]:
#Extract columns
column_lst=['Date','Location','Email','Visit ID','Date_Email_Location_Freq','Data Source','Leads Data Source','LEADID','Location Guest','Script To Insta','Script To Insta Based On Visit ID','Script To Insta Based On Date_Email_Location_Freq ID',
            'Insta To Leads','Insta To Leads Based On Visit ID','Insta To Leads Based On Date_Email_Location_Freq ID','Script To Leads','Script To Leads Based On Visit ID','Script To Leads Based On Date_Email_Location_Freq ID']
script_log_new_extract=script_log_new[column_lst]
instapage_new_extract=instapage_new[column_lst]
leads_new_extract=leads_new[column_lst]

In [92]:
#conbine 3 in 1
result = script_log_new_extract.append([instapage_new_extract,leads_new_extract], ignore_index=True)

In [93]:
#export csv
result.to_csv('final_ToLeads.csv')

In [94]:
result['Location Guest']

0                          NA
1                          NA
2                          NA
3                          NA
4                          NA
                 ...         
151105    2883279000012418035
151106    2883279000009870036
151107    2883279000012411012
151108    2883279000012411012
151109    2883279000012415025
Name: Location Guest, Length: 151110, dtype: object

# 3. Data check II (for Leads to CRM Contacts)
## 3.1 Calculation

In [95]:
contacts_new.head()

,CONTACTID,Email,Created Time,Location,Email Opt Out,User Type,User ID,Data Source
0,2883279000024240693,rejiveon@yahoo.com,2020-01-20,Unknown,No,Unique User,rejiveon@yahoo.com,CRM Contacts
1,2883279000024230708,gwendoss@att.net,2020-01-20,Unknown,No,Unique User,gwendoss@att.net,CRM Contacts
2,2883279000024232647,clarkeyc@gmail.com,2020-01-20,TrophyRoom-Chicago,No,Location User,clarkeyc@gmail.com - TrophyRoom-Chicago,CRM Contacts
3,2883279000024208720,bgrayer79@gmail.com,2020-01-20,CajunBoilandBar-OakPark,No,Location User,bgrayer79@gmail.com - CajunBoilandBar-OakPark,CRM Contacts
4,2883279000024206808,natoris43@gmail.com,2020-01-20,CajunBoilandBar-OakPark,No,Location User,natoris43@gmail.com - CajunBoilandBar-OakPark,CRM Contacts


In [96]:
contacts_new['User Type'].unique()

array(['Unique User', 'Location User', 'TappedIn Sales', 'Unknown'],
      dtype=object)

In [97]:
#Unique CONTACTID in Leads
leads_unique_CONTACTID_lst=leads_new['Location Guest'].unique().tolist()
#Unique Email in leads
leads_unique_email_lst=leads_new['Email'].unique().tolist()

In [98]:
#how many location user are from Leads
contacts_location_user=contacts_new.loc[contacts_new['User Type']=='Location User']
contacts_location_user_lst=contacts_location_user['CONTACTID'].tolist()
list_1 = [x for x in contacts_location_user_lst if x in leads_unique_CONTACTID_lst]
len(list_1)

17610

In [115]:
len(leads_unique_CONTACTID_lst)

18755

In [99]:
#craete 'Leads To Location User' column in Contacts dataset 
contacts_index_lst=[0] * contacts_new.shape[0]
for i in contacts_new.index:
    if (contacts_new['CONTACTID'][i] in list_1) and (contacts_new['User Type'][i]=='Location User'):
        contacts_index_lst[i]=1
contacts_new['Leads To Location User']=contacts_index_lst

In [100]:
##how many Unique user are from Leads
contacts_unique_user=contacts_new.loc[(contacts_new['User Type']=='Unique User')]
contacts_unique_user_lst=contacts_unique_user['Email'].tolist()
list_2 = [x for x in contacts_unique_user_lst if x in leads_unique_email_lst]
len(list_2)

17469

In [101]:
#craete 'Leads To Unique User' column in Contacts dataset 
contacts_index_lst=[0] * contacts_new.shape[0]
for i in contacts_new.index:
    if (contacts_new['Email'][i] in list_2) and (contacts_new['User Type'][i]=='Unique User'):
        contacts_index_lst[i]=1
contacts_new['Leads To Unique User']=contacts_index_lst

## 3.2 Datasets Combination of Leads and Contacts

In [102]:
leads_cb=leads_new.copy()

In [103]:
leads_cb=leads_cb[['Location Guest','Email','Date','Location','Data Source']]

In [104]:
leads_cb['Email Opt Out']='NA'
leads_cb['User Type']='NA'
leads_cb['User ID']='NA'
leads_cb['Leads To Location User']='NA'
leads_cb['Leads To Unique User']='NA'

In [105]:
leads_cb.shape

(74498, 10)

In [106]:
#rename
leads_cb=leads_cb.rename({"Location Guest":"CONTACTID","email":"Email"},axis='columns')
contacts_new=contacts_new.rename({"Created Time":"Date"},axis='columns')

In [108]:
#conbine 2 in 1
result2 = contacts_new.append([leads_cb], ignore_index=True)

C:\Users\Martin\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [109]:
#export csv
result2.to_csv('final_ToContacts.csv')

# 4. Data check II (for CRM Contacts to Campaign)
## 4.1 Calculation

In [ ]:
#how many Campaign contacts are from CRM
contacts_subscribers=contacts_new.loc[(contacts_new['User Type']=='Location User') & (contacts_new['Email Opt Out']=='No')]